<a href="https://colab.research.google.com/github/yiwenwangANU/pytorch_review/blob/main/Extra_Shakespeare_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
import os
from pathlib import Path
import requests

In [2]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("GPU ready!")
else:
  device = "cpu"
  print("GPU not available!")

GPU not available!


##Data download and explore

In [3]:
data_dir = Path.home() / 'data'
os.makedirs(data_dir, exist_ok=True)

data_dir = os.path.join(data_dir, "input.txt")
if os.path.exists(data_dir):
  print("Data ready.")
else:
  print(f"Downloading training data to {data_dir}")
  url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
  torch.hub.download_url_to_file(url, data_dir)

100%|██████████| 1.06M/1.06M [00:00<00:00, 16.9MB/s]


In [4]:
with open(data_dir, 'r', encoding='utf-8') as f:
    text = f.read()
    print(f"length of dataset in characters: {len(text)}")
    print('First 300 chars:')
    print(text[:300])

length of dataset in characters: 1115394
First 300 chars:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


##Tokenization

In [20]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print(f'Number of unique chars: {vocab_size}')
print(vocab)

Number of unique chars: 65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
charTotoken = {char:i for i,char in enumerate(vocab)}
tokenTochar = {i:char for i,char in enumerate(vocab)}

In [7]:
def encode(input):
  return [charTotoken[char] for char in input]

def decode(input):
  return ''.join([tokenTochar[token] for token in input])

In [8]:
print(encode('First Citizen:'))
print(decode(encode('First Citizen:')))

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10]
First Citizen:


Check other tokenizer like gpt-tokenizer

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


##Train test split

In [10]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [43]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_sample(batch_size, block_size):
  idxes = torch.randint(len(train_data)-block_size, (batch_size,))
  xb = torch.stack([train_data[idx:idx+block_size] for idx in idxes])
  yb = torch.stack([train_data[idx+1: idx+block_size+1] for idx in idxes])
  return xb, yb

xb, yb = get_sample(batch_size, block_size)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
targets:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
----
when input is [57] the target: 1
when input is [57, 1] the target: 46
when input is [57, 1, 46] the target: 47
when input is [57, 1, 46, 47] the target: 57
when input is [57, 1, 46, 47, 57] the target: 1
when input is [57, 1, 46, 47, 57, 1] the target: 50
when input is [57, 1, 46, 47, 57, 1, 50] the target: 53
when input is [57, 1, 46, 47, 57, 1, 50, 53] the target: 60
when input is [1] the target: 58
when input is [1, 58] the target: 46
when input is [1, 58, 46] the target: 43
when input is [1, 58, 46, 43] the target: 56
when input is [1, 58, 46, 43, 56] the target: 43
when input is [1, 58, 46,

In [18]:
embedding = nn.Embedding(num_embeddings=10, embedding_dim=3)

# Input tensor (indices of words or items)
input_indices = torch.tensor([1, 5, 7, 7])

# Get embeddings
output = embedding(input_indices)
print(output)

tensor([[ 1.1914, -0.8140, -0.7360],
        [-0.3387, -1.3407, -0.5854],
        [ 0.0516,  0.7440, -0.4816],
        [ 0.0516,  0.7440, -0.4816]], grad_fn=<EmbeddingBackward0>)


In [22]:
torch.manual_seed(42)

# A Bigram Model predicts the next token based only on the previous token
# (i.e., it learns a probability distribution of token transitions).
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
  def forward(self, x):
    return self.token_embedding_table(x)

model_bigram = BigramLanguageModel(vocab_size)

In [31]:
# use the last token in x to predict next token
# and loop to generate text at seq_len length
torch.manual_seed(42)
def generate(x, seq_len): # x-> shape(Batch size, 1)
  for _ in range(seq_len):
    logits = model_bigram(x) # shape(Batch size, Sequence length, Vocabulary size)
    logits_last = logits[:,-1,:] # shape(Batch size, Vocabulary size)
    logits_last_prob = F.softmax(logits_last, dim=-1) # shape(Batch size, Vocabulary size)
    token_next = torch.multinomial(logits_last_prob, num_samples=1) # shape(B, 1)
    x = torch.cat((x, token_next), dim=1) # (B, T+1)
  return x

In [49]:
# generate use untrained model to generate text
new_text = generate(torch.zeros(1, 1, dtype=torch.long), seq_len=100)
print(new_text)
print(decode(new_text.squeeze().numpy()))

tensor([[ 0, 53, 49, 22, 55, 59, 20,  7, 25, 56, 64, 54, 42, 62, 45, 49, 21, 58,
         16, 35, 55, 20, 39, 45, 56, 37,  9, 37, 49, 17, 30,  0, 61, 48, 15, 32,
          5, 32, 61,  0, 63, 54, 38,  5, 25, 18, 12, 35, 15, 24, 45, 46, 23, 40,
          0, 61, 52, 23, 54, 47, 48, 18,  2, 21,  7, 23, 54, 16, 54, 42, 49, 18,
         37, 12, 35, 41,  4, 49, 17, 11,  8, 60,  9, 62, 52, 41, 25, 13,  0, 55,
         52, 10, 14, 12, 12, 49, 51, 23,  5, 20, 33]])

okJquH-MrzpdxgkItDWqHagrY3YkER
wjCT'Tw
ypZ'MF?WCLghKb
wnKpijF!I-KpDpdkFY?Wc&kE;.v3xncMA
qn:B??kmK'HU


In [23]:
optimizer = torch.optim.Adam(params=model_bigram.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [53]:
epoches = 100
for epoch in range(epoches):
  model_bigram.train()
  xb, yb = get_sample(batch_size, block_size) # get random sample in each epoch
  logits = model_bigram(xb)
  B, T, C = logits.shape # change the shape to match the input shape of loss_fn
  logits = logits.view(B*T, C)
  targets = yb.view(B*T)
  loss = loss_fn(logits, targets)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if(epoch == 0):
    print(f'Logits shape: {logits.shape}')
  if(epoch % 20 == 0):
    print(f'epoch: {epoch}, train loss: {loss.item():.2f}')


Logits shape: torch.Size([32, 65])
epoch: 0, train loss: 4.61
epoch: 20, train loss: 4.18
epoch: 40, train loss: 4.37
epoch: 60, train loss: 4.36
epoch: 80, train loss: 4.33


In [54]:
# generate use trained model to generate text
print(decode(generate(torch.zeros(1, 1, dtype=torch.long), seq_len=100).squeeze().numpy()))


wVJTKvTwn'!MVxTVT'ryMq 3niU.WvoMFUr:JJc&eF$RIKQVaqK:mfB?pcNwEmDOtnxFjvTwyk- dGmRjYxDJPGIIMCa&?ZeOrBW
